In [1]:
#Upload your kaggle API json file:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pramodh941","key":"8a743bc4d1f00db1f052ef456a100e62"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d stackoverflow/stacksample

 99% 1.09G/1.11G [00:12<00:00, 105MB/s] 
100% 1.11G/1.11G [00:12<00:00, 95.3MB/s]


In [4]:
#Define the path and name of the directory to be created:
#This is the path where we will extract the dataset:
import os
path = "/content/SS"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory /content/SS 


In [5]:
#Extracts the Images from the Zip file downloaded to the path we created above:
from zipfile import ZipFile
file_name = "stacksample.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall(path = "/content/SS/")
  print('Done')

Done


In [6]:
#If you face issue loading the necessary java files:
!sudo apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

In [7]:
!sudo apt-get install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 68 not upgraded.


In [8]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

import findspark
findspark.init("spark-3.0.1-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

In [29]:
user_schema = 'Id INT,OwnerUserId INT,CreationDate TIMESTAMP,ClosedDate TIMESTAMP,Score INT,Title STRING,Body STRING'
df = spark.read.option("header","true").csv('/content/SS/Questions.csv', schema = user_schema, sep=',', dateFormat='MM/dd/yyyy',timestampFormat='yyyy-MM-dd HH:mm:ss.SSSSSS') #.limit(100000)

In [30]:
df.show()

+----+-----------+-------------------+----------+-----+--------------------+--------------------+
|  Id|OwnerUserId|       CreationDate|ClosedDate|Score|               Title|                Body|
+----+-----------+-------------------+----------+-----+--------------------+--------------------+
|  80|         26|2008-08-01 13:57:07|      null|   26|SQLStatement.exec...|"<p>I've written ...|
|null|       null|               null|      null| null|                null|                null|
|null|       null|               null|      null| null|                null|                null|
|null|       null|               null|      null| null|                null|                null|
|null|       null|               null|      null| null|                null|                null|
|null|       null|               null|      null| null|                null|                null|
|null|       null|               null|      null| null|                null|                null|
|null|       null|  

In [31]:
df.dtypes

[('Id', 'int'),
 ('OwnerUserId', 'int'),
 ('CreationDate', 'timestamp'),
 ('ClosedDate', 'timestamp'),
 ('Score', 'int'),
 ('Title', 'string'),
 ('Body', 'string')]

In [33]:
columns_to_drop = ['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate','Score', 'Title']
df = df.drop(*columns_to_drop)

In [34]:
df.show()

+--------------------+
|                Body|
+--------------------+
|"<p>I've written ...|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
|                null|
+--------------------+
only showing top 20 rows



In [ ]:
df.filter("Body is null").count()

31745443

In [35]:
df = df.filter(df.Body.isNotNull())

In [36]:
df.count()

1404281

In [52]:
from pyspark.sql.functions import split, col
df2 = df.withColumn("Body", split(col("Body"), "\s+").cast("array<string>")) #in cast change as required data type suitable for the methods we use

In [53]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="Body", outputCol="BodyMinusStopWords")
dfMinusStopWords = remover.transform(df2)

In [55]:
dfMinusStopWords.show()

+--------------------+--------------------+
|                Body|  BodyMinusStopWords|
+--------------------+--------------------+
|["<p>I've, writte...|["<p>I've, writte...|
|["<p>Are, there, ...|["<p>Are, really,...|
|[<p>Has, anyone, ...|[<p>Has, anyone, ...|
|[<p>This, is, som...|[<p>This, somethi...|
|[<p>I, have, a, l...|[<p>I, little, ga...|
|[<p>I, am, workin...|[<p>I, working, c...|
|[<p>I've, been, w...|[<p>I've, writing...|
|[<p>I, wonder, ho...|[<p>I, wonder, gu...|
|[<p>I, would, lik...|[<p>I, like, vers...|
|[<p>I'm, trying, ...|[<p>I'm, trying, ...|
|[, and, haven't, ...|[, seen, User/Mac...|
|[<p>What's, the, ...|[<p>What's, simpl...|
|[<p>I, need, to, ...|[<p>I, need, grab...|
|[<p>I'm, looking,...|[<p>I'm, looking,...|
|[<p>What, is, the...|[<p>What, correct...|
|[<p>I, am, using,...|[<p>I, using, CCN...|
|[<p>I, am, lookin...|[<p>I, looking, a...|
|[<p>I, am, using,...|[<p>I, using, MSB...|
|[<p>I'm, setting,...|[<p>I'm, setting,...|
|[<p>I, always, cr...|[<p>I, alw

In [69]:
dfMinusStopWords.dtypes

[('Body', 'array<string>'), ('BodyMinusStopWords', 'array<string>')]

In [71]:
from pyspark.sql.functions import concat_ws

In [72]:
df3 = dfMinusStopWords.withColumn("BodyMinusStopWords", concat_ws(",",col("BodyMinusStopWords")))

In [73]:
df3.dtypes

[('Body', 'array<string>'), ('BodyMinusStopWords', 'string')]

In [74]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
tokenizer = Tokenizer(inputCol="BodyMinusStopWords", outputCol="words")

In [75]:
wordsData = tokenizer.transform(df3)

In [76]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10)

In [77]:
featurizedData = hashingTF.transform(wordsData)

In [78]:
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [79]:
idfModel = idf.fit(featurizedData)

In [80]:
TfIdfData = idfModel.transform(featurizedData)

In [81]:
TfIdfData.show()

+--------------------+--------------------+--------------------+--------------+--------------------+
|                Body|  BodyMinusStopWords|               words|   rawFeatures|            features|
+--------------------+--------------------+--------------------+--------------+--------------------+
|["<p>I've, writte...|"<p>I've,written,...|["<p>i've,written...|(10,[0],[1.0])|(10,[0],[2.297344...|
|["<p>Are, there, ...|"<p>Are,really,go...|["<p>are,really,g...|(10,[1],[1.0])|(10,[1],[2.315494...|
|[<p>Has, anyone, ...|<p>Has,anyone,got...|[<p>has,anyone,go...|(10,[6],[1.0])|(10,[6],[2.273655...|
|[<p>This, is, som...|<p>This,something...|[<p>this,somethin...|(10,[1],[1.0])|(10,[1],[2.315494...|
|[<p>I, have, a, l...|<p>I,little,game,...|[<p>i,little,game...|(10,[6],[1.0])|(10,[6],[2.273655...|
|[<p>I, am, workin...|<p>I,working,coll...|[<p>i,working,col...|(10,[6],[1.0])|(10,[6],[2.273655...|
|[<p>I've, been, w...|<p>I've,writing,w...|[<p>i've,writing,...|(10,[5],[1.0])|(10,[5],[2.2

In [47]:
from pyspark.ml.feature import MinHashLSH

In [82]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=20)
LSHmodel = mh.fit(TfIdfData)

In [83]:
LSH = LSHmodel.transform(TfIdfData)

In [84]:
LSH.show()

+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+
|                Body|  BodyMinusStopWords|               words|   rawFeatures|            features|              hashes|
+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+
|["<p>I've, writte...|"<p>I've,written,...|["<p>i've,written...|(10,[0],[1.0])|(10,[0],[2.297344...|[[1.886299628E9],...|
|["<p>Are, there, ...|"<p>Are,really,go...|["<p>are,really,g...|(10,[1],[1.0])|(10,[1],[2.315494...|[[8.38810225E8], ...|
|[<p>Has, anyone, ...|<p>Has,anyone,got...|[<p>has,anyone,go...|(10,[6],[1.0])|(10,[6],[2.273655...|[[1.715587439E9],...|
|[<p>This, is, som...|<p>This,something...|[<p>this,somethin...|(10,[1],[1.0])|(10,[1],[2.315494...|[[8.38810225E8], ...|
|[<p>I, have, a, l...|<p>I,little,game,...|[<p>i,little,game...|(10,[6],[1.0])|(10,[6],[2.273655...|[[1.715587439E9],...|
|[<p>I, am, workin...|<p

In [ ]:
#Past work : needs editing
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=20, minCount=0, inputCol="_c6", outputCol="Features")

model = word2Vec.fit(df2)

In [ ]:
Features = model.transform(df2)
for row in Features.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Streaming output truncated to the last 5000 lines.
Text: [, when, printf(), is, okay, with, just, ""%f""?"] => 
Vector: [0.12044993272866122,0.04088372248224914,0.1443963183555752,-0.024615786009235308,0.0244286865927279,-0.20430144912097603,-0.018225584703031927,0.03543676680419594,0.02903681725729257,0.07480629545170814,0.08882052259286866,0.0416833731578663,-0.10798378131585196,0.09508046670816839,-0.09383531754428986,-0.007048223400488496,0.015048749482957646,-0.09434734494425356,-0.02630311535904184,0.00931498542195186]

Text: [<p>I, have, created, HTTP, handlers., </p>] => 
Vector: [0.0471714901116987,-0.15171759800675014,0.07742957879478732,-0.02439976887156566,-0.060513355458776154,-0.12671914137899876,0.1325740225147456,0.18171857545773187,-0.028452209041764338,-0.24148203991353512,0.2803177159900466,-0.024640841409564018,-0.10032574459910393,0.007714758937557538,0.1990256548548738,0.1265423195436597,0.15744307730468182,0.008536541795668501,-0.03541543489942948,-0.317262185815

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

